# [Advent of Code 2021](https://adventofcode.com/2021)

Partially inspired by [Norvig's pytudes](https://github.com/norvig/pytudes/blob/main/ipynb/Advent-2020.ipynb), although I probably won't be using the same style.

Goals
------
* Try to finish all gold stars on day of puzzle
* Learn new python features
* Investigate github copilot

In [1]:
# Imports

from collections import defaultdict
import numpy as np
import re
from typing import Callable, TypeVar
from functools import lru_cache

T = TypeVar('T')

# Helper functions

def data(day: int, parser: Callable[[str], T]) -> list[T]:
    with open(f"../data/2021/day{day}.txt") as f:
        return [parser(line.strip()) for line in f.readlines()]

## Day 1

Determine number of increments in list

In [13]:
data1 = data(1, int)

In [14]:
def day1(data: list[int]) -> int:
  count, current = 0, data[0]
  for i in data[1:]:
    if i > current:
      count += 1
    current = i
  return count

day1(data1)

1688

In [48]:
def win3(data: list[int]) -> list[int]:
  for i in range(len(data) - 2):
    yield sum(data[i:i+3])

day1(list(win3(data1)))

1728

## Day 2

Maintain sum of depth and length

In [21]:
data2 = data('2', str)

In [22]:
def day2(data: list[str]) -> int:
  depth, length = 0, 0
  for i in data:
    mode, amount = i.split(' ')
    amount = int(amount)
    match mode:
      case 'forward':
        length += amount
      case 'down':
        depth += amount
      case 'up':
        depth -= amount
  return depth * length

day2(data2)

1451208

In [26]:
def day2_2(data: list[str]) -> int:
  depth, length, aim = 0, 0, 0
  for i in data:
    mode, amount = i.split(' ')
    amount = int(amount)
    match mode:
      case 'forward':
        length += amount
        depth += amount * aim
      case 'down':
        aim += amount
      case 'up':
        aim -= amount
  return depth * length

day2_2(data2)

1620141160

## Day 3

Can probably be solved with bit tricks, but I'll just implement them as array problems.

In [176]:
data3 = data('3', str)

def binlist2int(l: list[int]) -> int:
  return int("".join(str(i) for i in l), 2)

In [177]:
def day3(data: list[int]):
  total = len(data)
  size = len(data[0])
  sums = [0] * size
  for binary in data:
    for i, v in enumerate(binary):
      sums[i] += int(v)
  
  gamma = [0 if total - i > i else 1 for i in sums]
  epsilon = [i ^ 1 for i in gamma]
  return binlist2int(gamma) * binlist2int(epsilon)

day3(data3)

2954600

In [179]:
def find_boundary(data: list[str], start: int, end: int, index: int) -> int:
  # TODO: this can be implemented as binary search
  for i, v in enumerate(data[start:end+1]):
    if v[index] == '1':
      return start + i
  return start

def scrubber(data: list[str], lcd: bool) -> int:
  # Run "binary search"
  low, high = 0, len(data) - 1
  for target in range(len(data[0])):
    index = low + ((high - low + 1) // 2)
    item = data[index][target]

    if lcd and item == '0':
      # Select all '1's in the bottom half
      low = find_boundary(data, index, high, target)
    elif lcd and item == '1':
      # Select all '0's in the top half
      high = find_boundary(data, low, index, target) - 1
    elif item == '0':
      # Select all '0's from the top through to the bottom half
      high = find_boundary(data, index, high, target) - 1
    else:
      # Select all '1' starting from the top half to the end
      low = find_boundary(data, low, index, target)
    
    if low == high:
      break

  return data[low]

def day3_2(data: list[str]) -> int:
  data = sorted(data)
  oxygen = scrubber(data, False)
  c02 = scrubber(data, True)
  return binlist2int(oxygen) * binlist2int(c02)

day3_2(data3)

1662846

## Day 4
Reference storing Bingo Boards

In [81]:
data4 = data('4', str)

class BingoBoard:
  def __init__(self):
    self.board = []
    self.size = 0
    # TODO: this state should be stored elsewhere
    self.rows = []
    self.cols = []
    
  def add_row(self, row: list[int]):
    self.board.append(row)
    if not self.size:
      self.size = len(row)
      self.rows = [0] * self.size
      # Assume column size is same as row size
      self.cols = [0] * self.size

def parse_data4(data: list[str]) -> tuple[list[int], list[BingoBoard]]:
  numbers = [int(x) for x in data[0].split(',')]
  boards, board = [], None
  for row in data[1:]:
    if row == '':
      board = BingoBoard()
      boards.append(board)
    else:
      board.add_row([int(x) for x in re.split('\s+', row)])
  return numbers, boards

def analyze_boards(boards: list[BingoBoard]) -> dict[list[tuple[int,int,BingoBoard]]]:
  # For each number, store the row and column with every board it was found in
  keys = defaultdict(lambda: [])
  for board in boards:
    for i, row in enumerate(board.board):
      for j, v in enumerate(row):
        keys[v].append((i, j, board))
  return keys

In [75]:
def calculate_score(board: BingoBoard, numbers: set[int]) -> int:
  score = 0
  for row in board.board:
    for v in row:
      if v not in numbers:
        score += v
  return score

def day4(data: list[str]) -> int:
  numbers, boards = parse_data4(data)  
  keys = analyze_boards(boards)
  selected = set()
  for number in numbers:
    if number in selected:
      continue
    selected.add(number)
    for i, j, board in keys[number]:
      board.rows[i] += 1
      board.cols[j] += 1
      if board.size == board.rows[i] or board.size == board.cols[j]:
        return calculate_score(board, selected) * number

day4(data4)

4512

In [82]:
def retroactive_selected(inputs: list[int], end: int) -> int:
  index = inputs.index(end) + 1
  return set(inputs[:index])

def day4_2(data: list[str]) -> int:
  numbers, boards = parse_data4(data)  
  keys = analyze_boards(boards)
  selected = set()
  completed, last_winner, last_num = set(), None, 0
  for number in numbers:
    if number in selected:
      continue
    selected.add(number)
    for i, j, board in keys[number]:
      if board in completed:
        continue
      board.rows[i] += 1
      board.cols[j] += 1
      if board.size == board.rows[i] or board.size == board.cols[j]:
        completed.add(board)
        last_winner, last_num = board, number
  return calculate_score(last_winner, retroactive_selected(numbers, last_num)) * last_num

day4_2(data4)

5586

## Day 5

Some possible ideas for solutions: 
1. use vector products between each pair (n^2) and remove duplicates somehow
2. use a 2-d interval tree and detect where sums on a point are greater than 1
3. fill a 2-d grid and iterate through points

Implementing with idea 3 for now, since size of grid is small relative to number of lines

In [102]:
Line = tuple[tuple[int, int], tuple[int, int]]

def parse5(line: str) -> list[Line]:
  result = re.split('[\s,]', line)
  return (
    (int(result[0]), int(result[3])), # xs
    (int(result[1]), int(result[4])) # ys
  )

data5 = data('5', parse5)

In [103]:
class BaseHydroVents:
  def __init__(self, size: int) -> None:
    self.grid = np.zeros([size, size])

  def add_line(self, line: Line) -> None:
    if line[0][0] == line[0][1]:
      self.add_vertical_line(sorted(line[1]), line[0][0])
    elif line[1][0] == line[1][1]:
      self.add_horizontal_line(sorted(line[0]), line[1][0])
    else:
      self.add_diagonal_line(line)

  def add_diagonal_line(self, line: Line) -> None:
    # Stub
    return

  def add_horizontal_line(self, xs: tuple[int, int], y: int) -> None:
    for x in range(xs[0], xs[1] + 1):
      self.grid[y][x] += 1

  def add_vertical_line(self, ys: tuple[int, int], x: int) -> None:
    for y in range(ys[0], ys[1] + 1):
      self.grid[y][x] += 1

def max_vent_point(data: list[Line]) -> int:
  return max([max(max(x[0]), max(x[1])) for x in data])

def day5(data: list[Line]) -> int:
  size = max_vent_point(data) + 1
  vents = BaseHydroVents(size)
  for line in data:
    vents.add_line(line)
  mask = vents.grid > 1
  return np.count_nonzero(mask)

day5(data5)


4728

In [104]:
class HydroVents (BaseHydroVents):
  def __init__(self, size: int) -> None:
    self.grid = np.zeros([size, size])

  def add_diagonal_line(self, line: Line) -> None:
    # Only supports 45 degree lines
    xs, ys = line[0], line[1]
    dx = 1 if xs[1] > xs[0] else -1
    dy = 1 if ys[1] > ys[0] else -1
    length = abs(xs[1] - xs[0]) + 1
    for i in range(length):
      self.grid[ys[0] + dy * i][xs[0] + dx * i] += 1

def day5_2(data: list[Line]) -> int:
  size = max_vent_point(data) + 1
  vents = HydroVents(size)
  for line in data:
    vents.add_line(line)
  mask = vents.grid > 1
  return np.count_nonzero(mask)

day5_2(data5)


17717

## Day 6

Seems like a DP problem

In [96]:
data6 = data('6', lambda x: [int(n) for n in x.split(',')])[0]

In [99]:
@lru_cache(None)
def spawns6(days: int):
  if days < 6:
    return 1
  return spawns6(days-7) + spawns8(days-7)

@lru_cache(None)
def spawns8(days: int):
  if days < 8:
    return 1
  return spawns6(days-9) + spawns8(days-9)

def day6(data: list[int], days: int) -> int:
  result = [spawns6(days + 6 - i) for i in data]
  return sum(result)

day6(data6, 79)

394994

In [100]:
day6(data6, 255)

1765974267455

## Day 7

TODO

In [101]:
# Scratch